# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [1]:
import pandas as pd

# Carregar o dataset
df = pd.read_csv('demo01.csv')

# Avaliar as variáveis e os tipos de dados
print("Informações sobre as variáveis e tipos de dados:")
print(df.info())

# Verificar a quantidade de valores ausentes
print("\nQuantidade de valores ausentes por coluna:")
print(df.isnull().sum())

# Distribuição da variável resposta 'mau'
print("\nDistribuição da variável resposta (mau):")
print(df['mau'].value_counts())


Informações sobre as variáveis e tipos de dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16650 entries, 0 to 16649
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   16650 non-null  object 
 1   posse_de_veiculo       16650 non-null  object 
 2   posse_de_imovel        16650 non-null  object 
 3   qtd_filhos             16650 non-null  int64  
 4   tipo_renda             16650 non-null  object 
 5   educacao               16650 non-null  object 
 6   estado_civil           16650 non-null  object 
 7   tipo_residencia        16650 non-null  object 
 8   idade                  16650 non-null  float64
 9   tempo_emprego          16650 non-null  float64
 10  possui_celular         16650 non-null  int64  
 11  possui_fone_comercial  16650 non-null  int64  
 12  possui_fone            16650 non-null  int64  
 13  possui_email           16650 non-null  int64  
 14  qt_pe

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [3]:
# Criar um DataFrame com os nomes das variáveis e os tipos de dados
meta_dados = pd.DataFrame({
    'nome_variavel': df.columns,
    'tipo_dado': df.dtypes.astype(str)
})

# Inicializar uma lista vazia para armazenar a quantidade de categorias de cada variável
qtd_categorias = []

# Iterar ao longo das variáveis e contar o número de categorias para cada uma
for coluna in df.columns:
    # Se a variável for categórica (object ou bool), contar as categorias
    if df[coluna].dtype == 'object' or df[coluna].dtype == 'bool':
        qtd_categorias.append(df[coluna].nunique())
    else:
        qtd_categorias.append(None)  # Variáveis não categóricas recebem None

# Adicionar a lista como uma nova coluna no DataFrame de metadados
meta_dados['qtd_categorias'] = qtd_categorias

# Exibir o DataFrame de metadados
print(meta_dados)

                               nome_variavel tipo_dado  qtd_categorias
sexo                                    sexo    object             2.0
posse_de_veiculo            posse_de_veiculo    object             2.0
posse_de_imovel              posse_de_imovel    object             2.0
qtd_filhos                        qtd_filhos     int64             NaN
tipo_renda                        tipo_renda    object             5.0
educacao                            educacao    object             5.0
estado_civil                    estado_civil    object             5.0
tipo_residencia              tipo_residencia    object             6.0
idade                                  idade   float64             NaN
tempo_emprego                  tempo_emprego   float64             NaN
possui_celular                possui_celular     int64             NaN
possui_fone_comercial  possui_fone_comercial     int64             NaN
possui_fone                      possui_fone     int64             NaN
possui

#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [5]:
# Identificar variáveis que precisam de dummies (variáveis qualitativas não numéricas)
# Excluir variáveis numéricas e booleanas
variaveis_categoricas = df.select_dtypes(include=['object']).columns

# Criar variáveis dummies para as variáveis categóricas
df_dummies = pd.get_dummies(df, columns=variaveis_categoricas, drop_first=True)

# Eliminar as variáveis tipo string e manter apenas as versões dummy no DataFrame final
df_final = df_dummies

# Exibir o DataFrame final apropriado para o scikit-learn
print(df_final.head())

   qtd_filhos      idade  tempo_emprego  possui_celular  \
0           0  58.832877       3.106849               1   
1           0  52.356164       8.358904               1   
2           0  52.356164       8.358904               1   
3           0  46.224658       2.106849               1   
4           0  29.230137       3.021918               1   

   possui_fone_comercial  possui_fone  possui_email  qt_pessoas_residencia  \
0                      0            0             0                    2.0   
1                      0            1             1                    1.0   
2                      0            1             1                    1.0   
3                      1            1             1                    2.0   
4                      0            0             0                    2.0   

     mau  sexo_M  ...  educacao_Secondary / secondary special  \
0  False    True  ...                                    True   
1  False   False  ...                         

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [15]:
# Criar uma tabela cruzada entre 'possui_email', 'posse_de_veiculo' e a variável resposta 'mau'
tabela_cruzada = pd.crosstab(index=df['possui_email'], columns=df['posse_de_veiculo'], values=df['mau'], aggfunc='mean')

# Exibir a tabela cruzada
print("Tabela cruzada entre possui_email, posse_de_veiculo e a proporção de 'mau = 1':")
print(tabela_cruzada)
# A variável possui_email parece ser mais importante para prever a inadimplência, já que quem não tem email tem uma taxa maior de "mau".

Tabela cruzada entre possui_email, posse_de_veiculo e a proporção de 'mau = 1':
posse_de_veiculo         N         Y
possui_email                        
0                 0.024944  0.021788
1                 0.016726  0.024883


#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [13]:
# Salvar o DataFrame preparado para uso posterior
df_final.to_csv('base_preparada_para_scikitlearn.csv', index=False)